In [41]:
import pandas as pd
import numpy as np

artist_data = pd.read_csv('datasets/ydata-ymusic-user-artist-rating.csv')
songs_data = pd.read_csv('datasets/ydata-ymusic-user-song-rating.csv')
artist_name_data = pd.read_csv('datasets/ydata-ymusic-artist-names.csv')

In [42]:
#Data preprocessing
def data_preprocessing(artist_data, groupByVar, indexId):
    
    #to show popular artist
    avg_rating = pd.DataFrame(artist_data.groupby(groupByVar)['rating'].mean().reset_index())
    avg_rating.sort_values('rating', ascending = False).head(20)
    
    #creating pivot table
    ratedBygroupByVar = pd.pivot_table(data=artist_data, values='rating', index=indexId, columns=groupByVar)
    
    return ratedBygroupByVar

In [43]:
def data_cleaning(recommended_data):
    
     #data cleaning and sorting
        
    recommended_corr = pd.DataFrame(recommended_data, columns=['Correlation'])
    recommended_corr.dropna(inplace=True)
    recommended_corr = recommended_corr.sort_values('Correlation', ascending=False).reset_index()
    
    return recommended_corr

In [44]:
def get_top_n_artist(recommended_corr, groupByVar, n, filteringFrom):
    
    #get top n artist list and artist names
    recommended_list = recommended_corr.head(n)[groupByVar].tolist()
    filter_names = filteringFrom[filteringFrom[groupByVar].isin(recommended_list)]
    
    return filter_names

In [45]:
def collaborative_filtering(data, groupByVar, indexId, Id, n, filteringFrom):
    
    #calling data preprocessing
    rating = data_preprocessing(data, groupByVar, indexId)
    
    #artist_id 
    #see groupByVar rated by indexId
    groupByVar_indexId_rating = rating[Id]
    
    #applying correlation for the entire user rating dataframe
    #Collaborative Filtering Song Recommendation
    #Pearson Correlation
    recommended_data = rating.corrwith(groupByVar_indexId_rating)
    
    #data cleaning and sorting
    recommended_corr = data_cleaning(recommended_data)
    
    #get top n artist list and artist names
    if(filteringFrom is None):
        recommended_items = recommended_corr.head(n)[groupByVar].tolist()
    else:
        recommended_items = get_top_n_artist(recommended_corr, groupByVar, n, filteringFrom)
    
    return recommended_items

In [46]:
recommended_artist_names = collaborative_filtering(artist_data,'artist_id', 'user_id', 1058037, 5, artist_name_data)
recommended_artist_names

/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,artist_id,artist
15486,1015911,Lords Of Acid
17287,1017742,Melvins
34356,1035342,Borknagar
39172,1040238,Mindless Self Indulgence
39633,1040701,Speak No Evil


In [ ]:
recommended_songs_Ids = collaborative_filtering(songs_data, 'song_id', 'user_id', 90134, 5, None)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
recommended_songs_Ids

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# calculate the correlation matrix
corr_matrix = artist_data.pivot_table(index='user_id', columns='artist_id', values='rating').corr()

# visualize the correlation matrix using a heatmap
plt.figure(figsize=(15,15))
plt.imshow(corr_matrix, cmap='coolwarm')
plt.colorbar()
plt.show()